In [ ]:
!pip3 install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

In [1]:
import json
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim


from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import AddedToken, GPT2Config, GPT2Tokenizer, GPT2ForSequenceClassification, GPT2Model
from text_utils import custom_data_gen, DBpedia, TextProcessor, TextTokenizer
from tqdm.autonotebook import tqdm
from utils import META_PATH

In [2]:
LR = 1e-5
N_EPOCHS = 5
BATCH_SIZE = 64
MAX_LEN = 100
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({
    "pad_token": AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True)
})

0

In [ ]:
## to do
## re split: part of train use for quantization, part of test use for examples

## just change contents of csv

__Plan__

__1.1. re split data 19:05 DONE__
  
__1.2. new dataloaders 19:20 DONE__

__2. write train loop 19:40__

__3. train gpt2 several attention layers 20:00__

__1. re split data__

In [ ]:
pwd

In [ ]:
# data = DBpedia(root="/root/quantized_nn/dbpedia_data")

# train_data = pd.read_csv("dbpedia_data/dbpedia_csv/train.csv", header=None)
# test_data = pd.read_csv("dbpedia_data/dbpedia_csv/test.csv", header=None)

# train_data.columns = ["class", "header", "content"]
# test_data.columns = ["class", "header", "content"]



# train_data, quant_data = train_test_split(train_data,
#                                           test_size=0.2,
#                                           stratify=train_data["class"],
#                                           random_state=1234567890)
# test_data, infer_data = train_test_split(test_data,
#                                          test_size=0.5,
#                                          stratify=test_data["class"],
#                                          random_state=1234567890)

# os.mkdir("data_db/")
# train_data.to_csv("data_db/train.csv", index=None)
# quant_data.to_csv("data_db/quant.csv", index=None)
# test_data.to_csv("data_db/test.csv", index=None)
# infer_data.to_csv("data_db/infer.csv", index=None)

__2. NEW DATALOADERS__

In [ ]:
# data_dir = "data_db/"

In [5]:
train_data = pd.read_csv("data_db/train.csv")
quant_data = pd.read_csv("data_db/quant.csv")
test_data = pd.read_csv("data_db/test.csv")
infer_data = pd.read_csv("data_db/infer.csv")
tst_0_data = train_data.iloc[:320]
tst_1_data = train_data.iloc[320:2*320]

In [6]:
data_train = ((row[0], row[2]) for row in train_data.values)
data_quant = ((row[0], row[2]) for row in quant_data.values)
data_test = ((row[0], row[2]) for row in test_data.values)
data_infer = ((row[0], row[2]) for row in infer_data.values)
data_tst_0 = ((row[0], row[2]) for row in tst_0_data.values)
data_tst_1 = ((row[0], row[2]) for row in tst_1_data.values)

In [ ]:
k = 0

for text, lab in quant_loader:
    print(text.size())
    print(lab.size())
    k += 1
    if k == 100:
        break

In [ ]:
labs_train = {i: 0 for i in range(1, 15)}
labs_test = {i: 0 for i in range(1, 15)}

for lab, _ in train_data:
    labs_train[lab] += 1
    
for lab, _ in test_data:
    labs_test[lab] += 1

In [28]:
gpt2_config = GPT2Config(vocab_size=len(tokenizer.get_vocab()))
setattr(gpt2_config, "pad_token_id", 50256)
model = GPT2ForSequenceClassification.from_pretrained(pretrained_model_name_or_path="gpt2",
                                                      config=gpt2_config)

new_score = nn.Sequential(
    nn.Linear(in_features=768, out_features=14, bias=False),
    nn.Sigmoid()
)

setattr(model, "score", new_score)



Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
model.load_state_dict(torch.load("gpt2_dbpedia_2305.pt"))

<All keys matched successfully>

In [30]:
num_layers = sum(1 for p in model.parameters())
print(f"model has {num_layers} layers")

trainable_num = 12

for num, (nam, p) in enumerate(model.named_parameters()):
    if num > num_layers - trainable_num:
        p.requires_grad = True
    else:
        p.requires_grad = False

model has 149 layers


In [31]:
model = model.to(DEVICE)

In [ ]:
for text, lab in train_loader:
    print(torch.argmax(model(text).logits, dim=-1))
    break

In [ ]:
data_tst_0 = ((row[2], row[0]) for row in tst_0_data.values)
data_tst_1 = ((row[2], row[0]) for row in tst_1_data.values)

test_ldr_0 = custom_data_gen(data_tst_0,
                               tokenizer,
                               MAX_LEN,
                               BATCH_SIZE,
                               DEVICE)

test_ldr_1 = custom_data_gen(data_tst_1,
                               tokenizer,
                               MAX_LEN,
                               BATCH_SIZE,
                               DEVICE)

In [32]:
class Trainer:
    
    def __init__(self,
                 model,
                 train_data,
                 test_data,
                 batch_size,
                 n_epochs,
                 plateau,
                 lr,
                 lr_decay,
                 device,
                 res_path,
                 loss_fn=None,
                 ):
        self.model = model
        self.train_data = train_data
        self.test_data = test_data
        self.batch_size = batch_size
        self.n_epochs = n_epochs
        self.plateau = plateau
        self.lr = lr
        self.lr_decay = lr_decay
        self.device = device
        self.res_path = res_path
        
        self.optimizer = optim.Adam(self.model.parameters(), self.lr)
        self.loss_fn = nn.CrossEntropyLoss()
        if loss_fn:
            self.loss_fn = getattr(nn, loss_fn)()
        self.history = {}
    
    def _dump_history(self):
        with open(self.res_path, "w") as f:
            json.dump(self.history, f)
    
    def _update_optimizer(self):
        print(f"LR reduced on plateau from {self.lr} to {self.lr*self.lr_decay}")
        self.lr *= self.lr_decay
        self.optimizer = optim.Adam(self.model.parameters(), self.lr)
    
    def train(self):
        self.history = {
            "loss": [],
            "acc": [],
            "best_loss": 1e10
        }
        no_gain = 0
        for epoch in range(1, self.n_epochs+1):
            loss, acc = self.train_epoch(epoch)
            if float(loss) < self.history["best_loss"]:
                self.history["best_loss"] = float(loss)
                no_gain = 0
            else:
                no_gain += 1
            if no_gain == self.plateau:
                self._update_optimizer()
            self.history["loss"].append(float(loss))
            self.history["acc"].append(float(acc))
            self._dump_history()
        return self.model
            
    def train_epoch(self,
                    epoch):
        loss_trn = 0
        batch_trn = 0
        loss_val = 0
        acc_val = 0
        batch_val = 0
        self.model.to(self.device)
        self.model.train()
        data_tst_0 = ((row[0], row[2]) for row in self.train_data.values)
        data_tst_1 = ((row[0], row[2]) for row in self.test_data.values)
        for x, y in tqdm(custom_data_gen(data_tst_0,
                                         tokenizer,
                                         MAX_LEN,
                                         self.batch_size,
                                         self.device)):
            loss = self.run_batch(x,
                                  y)
            batch_trn += 1
            loss_trn += loss.item()
        
        for x, y in tqdm(custom_data_gen(data_tst_1,
                                         tokenizer,
                                         MAX_LEN,
                                         self.batch_size,
                                         self.device)):
            loss, acc = self.run_batch(x,
                                       y,
                                       train=False)
            batch_val += 1
            loss_val += loss.item()
            acc_val += acc.item()
        
        print(f"Epoch {epoch}: val.loss - {loss_val / batch_val}, val.acc - {acc_val / batch_val}")
        return loss_val / batch_val, acc_val / batch_val
    
    def run_batch(self,
                  x,
                  y,
                  train=True):
        x = x.to(self.device)
        y = y.to(self.device)
        self.optimizer.zero_grad()
        preds = self.model(x).logits
        loss = self.loss_fn(preds, y.long())
        if train:
            loss.backward()
            self.optimizer.step()
            return loss
        else:
            acc = torch.sum(torch.argmax(preds, dim=-1) == y) / self.batch_size
            return loss, acc
    

In [33]:
trainer = Trainer(model,
                  train_data,
                  test_data,
                  BATCH_SIZE,
                  n_epochs=20,
                  plateau=3,
                  lr=LR,
                  lr_decay=0.8,
                  device=DEVICE,
                  res_path="results_gpt2_2405_morning.json")

In [34]:
model_trained = trainer.train()

0it [00:00, ?it/s]

KeyboardInterrupt: 

In [ ]:
import torch

In [ ]:
torch.save(model_trained.state_dict(), "gpt2_dbpedia_2305.pt")

In [ ]:
## add lr reduction on plateau

In [ ]:
x == 2

In [ ]:
GPT2Block

__transformers search__

1. models/gpt2/modeling_gpt2.py class GPT2ForSequenceClassification(GPT2PreTrainedModel)
2. models/gpt2/modeling_gpt2.py class GPT2PreTrainedModel(PreTrainedModel)
3. modeling_utils.py class PreTrainedModel(nn.Module, ModuleUtilsMixin, GenerationMixin, PushToHubMixin)
4. modeling_utils.py class ModuleUtilsMixin
5. generation_utils.py class GenerationMixin
6. file_utils.py class PushToHubMixin
7. models/gpt2/modeling_gpt2.py class GPT2Model(GPT2PreTrainedModel) (maybe embedding quantization can be ussed)
8. models/gpt2/modeling_gpt2.py class GPT2Block(nn.Module)
9. models/gpt2/modeling_gpt2.py class GPT2Attention(nn.Module) (__modify this to quantize attention block__)

In [ ]:
all_params = sum(torch.mul(*p.size()) for p in model.parameters() if len(p.size()) > 1).item()

In [ ]:
all_params

In [ ]:
768*768

In [ ]:
torch.mul(*torch.Tensor([768, 768]))

In [ ]:
x = p.size()

In [ ]:
torch.mul(*x).item()